In [22]:
#necessary python packages
import numpy as np
import os
import urllib.request
import pandas as pd 
import re

print("import of packages successful")

import of packages successful


In [23]:
# # Navigate to data
# os.chdir("/Users/ssaiya/chatbot/raw-data")

# # print the current working directory
# os.getcwd()

# Data Preperation
- Whatsapp Data is formatted in the following way: 
     {Date}, {Time} - {Author}: {Message}
- long messages can be multiline messages
- all messages will start with a timestamp

- info messages like group creation, adding members, removing members, and group name change do not have an author

In [24]:
# Helper method using regex to see if a line starts with a date or is a part of a long multiline message
def startsWithDate(s):
    pattern = '^(([0-9])|((1)[0-2]))(\/)(([0-2][0-9]|(3)[0-1])|([0-9]|(3)[0-1]))(\/)(\d{2}|\d{4}), (([0-9][0-9]):([0-9][0-9])|([0-9]):([0-9][0-9])) \s*([AP]M?) -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [25]:
# Helper method using regex to see if line has a message author or is an info message
# if this returns true then we can extract author name, else set it as None/null
def isHumanMessage(s):
#     print(s)
    patterns = [
        '([\w]+):',                              # First Name
        '([\w]+[\s]+[\w]+):',                    # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',          # First Name + Middle Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',          # First Name + Middle Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+[\s]+[\w]+):', # First Name + Middle Name + Last Name + extention
        '([\w]+[\s]+\'+[\w]+\'+[\s]+[\w]+):',    # First Name + 'Pet Name' + Last Name
        '([+]\d{2} \d{5} \d{5}):',               # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',         # Mobile Number (US)
        '([+]\d{2} \d{4} \d{7})'                 # Mobile Number (Europe)
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
#         print('MATCHED')
        return True
    return False

### Possible formats of messages:
- {date}, {time} - {Message} [this is used to give info messages about the group like name change, member change, etc]
- {date}, {time} - {Author} : {Message}

In [26]:
# Extract and tokenize messages 
def getDataLine(line):
    splitLine = line.split(' - ')
        
    dateTime = splitLine[0] # this should give {9/2/18, 9:15 PM}
    date, time = dateTime.split(',') # split into date and time 
    
    # date  - dd/mm/yy, 
    # time - hh:mm AM/PM [12 hour format]
    
    messageRaw = ' '.join(splitLine[1:])
    # this gives the message, now we check if its from an author or an info message
    
    if isHumanMessage(messageRaw):
#         print(messageRaw)
        messageArr = messageRaw.split(': ')
        author = messageArr[0]
        messageText = ' '.join(messageArr[1:])
    else:
        author = None
        messageText = messageRaw
    
    return date,time, author, messageText

In [27]:
# Now we can read the full text file, parse it while tokenizing the data into a tabular format

# import the raw chat as a pd dataframe with the help of our helper methods
parsedData = [] # List of data while parsing to be converted into a dataframe
testChatName = 'leagueChat.txt' # TODO this can be more modular in the future
pathToRawData = '/Users/ssaiya/chatbot/raw-data/'
pathToData = pathToRawData + testChatName
print('Opening file to read from path- \'' + pathToData+'\'')

with open(pathToData, encoding="utf-8") as fp:
    # Skipping first line of the file (usually contains information about end-to-end encryption)
    fp.readline()
    buffer = [] # Buffer to capture intermediate output for multi-line messages
    # variables to keep track of the current message being processed
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        
        if not line: # Stop reading further if end of file has been reached
            break
            
        line = line.strip() # remove trailing whitespaces
        
        # if a message starts with a timestamp it must be the begining of a message
        if startsWithDate(line):
#             print(line)
            if len(buffer) > 0:
                parsedData.append([date, time, author, ' '.join(buffer)]) #only point of entry for data
            buffer.clear() # since this is a new message clear the buffer which has been added to dataframe
            date, time, author, messageText = getDataLine(line)
#             print(messageText)
            buffer.append(messageText)
        # else it is part of a long multiline message so we can append it to message buffer    
        else :
            buffer.append(line)

Opening file to read from path- '/Users/ssaiya/chatbot/raw-data/leagueChat.txt'


In [29]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
df[:30]

,Date,Time,Author,Message
0,8/22/18,11:54 PM,None,"Menon created group ""Lovad"""
1,8/22/18,11:54 PM,None,+91 93265 97938 was added
2,9/2/18,9:15 PM,Liam Pereira,Has anybody been on the league subreddit
3,9/2/18,9:15 PM,Liam Pereira,Holy shit.
4,9/2/18,9:19 PM,Liam Pereira,<Media omitted>
5,9/2/18,9:20 PM,Liam Pereira,<Media omitted>
6,9/2/18,9:20 PM,Liam Pereira,"Riot, you done fucked up"
7,9/2/18,9:36 PM,Anand Merchant,Lmfao
8,9/2/18,9:36 PM,Anand Merchant,First JEE now RITO too
9,9/2/18,9:36 PM,Anand Merchant,Get this wamen reservation shit outta here


# Data points to explore
- most used emoji
- most active user as a function of frequency over time
- most messages sent
- least active users
- timeline with activity